In [ ]:
import os
os.chdir('../src')

### load getPretrained module

In [ ]:
%run modules/model/getPretrained.py

In [ ]:
ModelFileDict = getModelFileDict()
ModelFileDict

In [ ]:
modelwTop = modelImageNet('VGG16_includeTop', ModelFileDict['VGG16_includeTop'])
modelwTop.summary()

In [ ]:
outputTensorBoard('vgg16Folder_withTop')

In [ ]:
modelwTop.layers

In [ ]:
outputFromMaxPooling2D = modelwTop.layers[-5].output

In [ ]:
print('flatten layer', modelwTop.layers[-4].output)
print('dense layer',   modelwTop.layers[-3].output)

In [ ]:
with tf.variable_scope('additionalLayer'):
    flattenLayer  = tf.layers.Flatten(data_format='channels_last', name='flattenLayer')
    flattenOutput = flattenLayer(outputFromMaxPooling2D)

In [ ]:
outputTensorBoard('VGG16_changed')

In [ ]:
with tf.variable_scope('anotherLayer'):
    denseLayer = tf.layers.Dense(units=4096, activation=tf.nn.relu, name='denseLayer')
    output     = denseLayer(flattenOutput)

In [ ]:
outputTensorBoard('VGG16_changed_v2')

After changing layers, summary remain as the original

In [ ]:
modelwTop.summary()

In [ ]:
modelNew = keras.models.Model(inputs=modelwTop.input, outputs=output)

In [ ]:
modelNew.compile(optimizer='adam', loss='categorical_crossentropy')

In [ ]:
modelNew.summary()